In [8]:
#export
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

## Very good reading on inizialition! 

 - https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
 
- https://pouannes.github.io/blog/initialization/?fbclid=IwAR0lqZFCUUedF5edQS8z-RdsfqmI7fth-gk6ePIvHa2SChHNX6IVvr_oeAM

- https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables

In [9]:
def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

def test_near(a,b): 
    test(a,b,near)

In [10]:
def get_data(): 
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _)= pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train, y_train ,x_valid, y_valid))


def normalize(x, m, s): 
    return (x-m)/s


In [11]:
x_train,y_train,x_valid,y_valid = get_data()


In [12]:
train_mean,train_std = x_train.mean(), x_train.std() 

In [13]:
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [14]:
x_train = normalize(x_train, train_mean, train_std)
# Important here is that we use the train_mean and train_std instead of the validation
x_valid = normalize(x_valid, train_mean, train_std)


In [15]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [16]:
assert train_mean<0.01

In [17]:
n,m = x_train.shape
c = y_train.max()+1

In [18]:
n,m,c

(50000, 784, tensor(10))

In [19]:

#export
def test_near_zero(a,tol=1e-3): 
    assert a.abs()<tol, f"Near zero: {a}"

5000 rows, with 784 long tensors 28,28 images and 10 diffrent classes! 

## Foundations vesrison 

### Basic architecture

In [20]:
# nh nbr of hidden units 

nh = 50 

In [21]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [22]:
w1.shape

torch.Size([784, 50])

In [23]:
w1.std()

tensor(0.0357)

In [24]:
w1.mean()

tensor(0.0003)

In [25]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [26]:
x_valid.mean(),x_valid.std()


(tensor(-0.0057), tensor(0.9924))

In [27]:
def lin(x,w,b): 
    return x@w+b

In [28]:
t  = lin(x_valid, w1, b1)

In [29]:
t.shape

torch.Size([10000, 50])

In [30]:
t.mean(),t.std()

(tensor(0.0698), tensor(0.9851))

In [31]:
def relu(x):
    return  x.clamp_min(0.)

In [32]:
t = relu(lin(x_valid, w1, b1))

In [33]:
t.mean(), t.std()

(tensor(0.4263), tensor(0.6030))

# pytorch text docs! 

In [34]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)


In [35]:
w1.mean(),w1.std()


(tensor(0.0003), tensor(0.0505))

In [36]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5780), tensor(0.8526))

In [37]:
#export
from torch.nn import init


In [38]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()


(tensor(0.4721), tensor(0.7546))

In [39]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std})` where

    .. math::
        \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_ou

In [40]:
w1.mean(),w1.std()

(tensor(-0.0002), tensor(0.0506))

In [41]:
t.mean(),t.std()


(tensor(0.4721), tensor(0.7546))

In [42]:
w1.shape

torch.Size([784, 50])

In [43]:
import torch.nn

In [44]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [45]:
torch.nn.Linear.forward??

Signature: torch.nn.Linear.forward(self, input)
Source:   
    @weak_script_method
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
File:      ~/anaconda3/lib/python3.6/site-packages/torch/nn/modules/linear.py
Type:      function


In [46]:
torch.nn.functional.linear??

Signature: torch.nn.functional.linear(input, weight, bias=None)
Source:   
@weak_script
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret
File:      ~/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py
Type:      function


In [47]:
torch.nn.Conv2d??

Init signature: torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
Source:        
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.

    * :attr:`stride` controls the stride for the cross-correlation, a singl

In [48]:
def relu(x): 
    return x.clamp_min(0.)-0.5

In [49]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0906), tensor(0.8089))

In [50]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [51]:
%timeit -n 10 _=model(x_valid)
out1=model(x_valid)


10 loops, best of 3: 10.1 ms per loop


In [52]:
out1.shape

torch.Size([10000, 1])

## Loss Functions: MSE 

In [53]:
model(x_valid).shape

torch.Size([10000, 1])

In [54]:
def mse(output,targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [55]:
y_train,y_valid = y_train.float(),y_valid.float()


In [56]:
preds = model(x_train)


In [57]:
preds.squeeze(-1)

tensor([-0.2482, -0.7703, -1.9830,  ...,  0.8371, -1.2457,  0.0597])

In [58]:
mse(preds, y_train)


tensor(32.8664)

In [59]:
y_train

tensor([5., 0., 4.,  ..., 8., 4., 8.])

## Gradiets and backwards pass

For further information about the gradeient of the MSE see this link: https://mccormickml.com/2014/03/04/gradient-descent-derivation/

In [60]:
def mse_grad(inp,targ): 
    inp.g = 2. * (inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0] 
    
# OK 

In [61]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    # inp > 0 means if tha value is bigger then the value we check it get's a 1. Otherwise is zero
    inp.g = (inp>0).float() * out.g
    
#derivitive of a ax+b conserning x is a
# OK

In [65]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [66]:
from IPython import embed
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [67]:
forward_and_backward(x_train, y_train)

In [68]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [69]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [70]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [71]:
loss = forward(xt2, y_train)

In [72]:
loss.backward() # This calculates all the grads wer need!!!!!! Now i understand!! 

In [73]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Refactor model


In [ ]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [121]:
class Relu(): 
    def __call__(self, inp): 
        self.inp = inp 
        self.out = inp.clamp_min(0.)-0.5
        return self.out
        
    def backward(self): 
        self.inp.g = (self.inp>0).float()*self.out.g 

In [129]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b

        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [123]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [124]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [126]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [127]:
%time loss = model(x_train, y_train)


1
1
CPU times: user 153 ms, sys: 6.12 ms, total: 159 ms
Wall time: 61.4 ms


In [128]:
%time model.backward()


CPU times: user 5.39 s, sys: 2.47 s, total: 7.86 s
Wall time: 7.89 s


In [ ]:
# Module.forward 